In [27]:
!pip install transformers torchaudio jiwer


In [28]:
import os
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, logging
from jiwer import wer
import torch
import concurrent.futures
from tqdm.notebook import tqdm


In [29]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import tarfile
import os

# Define the path to the tar file
tar_path = '/content/drive/My Drive/VIKRAM-DRDO/nptel-pure-set.tar.gz'

# Define the extraction path
extraction_path = '/content/nptel-pure-set'

# Create the directory if it doesn't exist
if not os.path.exists(extraction_path):
    os.makedirs(extraction_path)

# Extract the tar file
with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(path=extraction_path)

print('Extraction completed.')


Extraction completed.


In [31]:
def load_audio_files(wav_directory, txt_directory):
    dataset = []
    for filename in tqdm(os.listdir(wav_directory), desc="Loading files"):
        if filename.lower().endswith('.wav'):
            audio_path = os.path.join(wav_directory, filename)
            transcript_path = os.path.join(txt_directory, filename.replace('.wav', '.txt'))
            try:
                waveform, sample_rate = torchaudio.load(audio_path)
                with open(transcript_path, 'r') as file:
                    transcript = file.read().strip()
                if transcript:
                    dataset.append((waveform, sample_rate, transcript))
            except Exception as e:
                print(f"Failed to load or process file {filename}: {e}")
    print(f"Loaded {len(dataset)} valid audio files.")
    return dataset


In [32]:
def speech_to_text(model, processor, waveform, sample_rate):
    if waveform.ndim > 1:
        waveform = waveform.mean(dim=0)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = inputs.input_values.to(model.device)

    with torch.no_grad():
        logits = model(inputs).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription


In [33]:
def evaluate_model_parallel(dataset, model, processor):
    references = [transcript.lower() for _, _, transcript in dataset]
    predictions = []

    # Define a small function to pass to the executor
    def process_audio(data):
        waveform, sample_rate, _ = data
        return speech_to_text(model, processor, waveform, sample_rate).lower()

    # Using ThreadPoolExecutor to parallelize processing
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(process_audio, dataset), total=len(dataset), desc="Processing audio"))

    predictions.extend(results)

    if predictions:
        error_rate = wer(references, predictions)
        print(f"Word Error Rate (WER): {error_rate:.6f}")
    else:
        print("No valid data to evaluate.")


In [34]:
# Set paths to your data
base_path = '/content/nptel-pure-set/nptel-pure'
wav_directory = os.path.join(base_path, 'wav')
txt_directory = os.path.join(base_path, 'corrected_txt')

# Reduce verbosity of transformers library to error only
logging.set_verbosity_error()

# Load dataset
dataset = load_audio_files(wav_directory, txt_directory)

# Load pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.to('cpu')  # Use 'cuda' if running on GPU

# Evaluate the model in parallel
evaluate_model_parallel(dataset, model, processor)


Loading files:   0%|          | 0/1000 [00:00<?, ?it/s]

Loaded 998 valid audio files.


Processing audio:   0%|          | 0/998 [00:00<?, ?it/s]

Word Error Rate (WER): 0.274513
